In [1]:
import pandas as pd


In [5]:
df = pd.concat([pd.read_csv("data/firearm-charges-2018.csv"),
                pd.read_csv("data/firearm-charges-2013.csv")]).dropna(subset=["Year"])

df

,Year,Charges,Convictions,Not Guilty,Nolle,Offense,Table
0,2013,4,0,0,4,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1
1,2014,3,0,0,3,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1
2,2015,5,2,0,3,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1
3,2016,7,2,0,5,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1
4,2017,10,0,0,10,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1
5,2013,1,0,0,1,FAILING TO SURRENDER REVOKED PERMIT TO CARRY A...,1
6,2013,100,36,0,64,ILLEGALLY TRANSFERRING A PISTOL OR REVOLVER (C...,1
7,2014,75,23,0,52,ILLEGALLY TRANSFERRING A PISTOL OR REVOLVER (C...,1
8,2015,116,57,0,59,ILLEGALLY TRANSFERRING A PISTOL OR REVOLVER (C...,1
9,2016,77,17,0,60,ILLEGALLY TRANSFERRING A PISTOL OR REVOLVER (C...,1


In [6]:
def asciistr(badstr):
    if pd.isnull(badstr): return badstr
    return ''.join([i if ord(i) < 128 else '' for i in badstr]).strip().replace("/"," ")
df["ascii_offense"] = df["Offense"].apply(asciistr)

In [7]:
import re
def statute_code(offense_str):
    if pd.isnull(offense_str): return ""
    return re.search(".* \((.*)\)",asciistr(offense_str)).group(1)

statute_code(df.iloc[20]["Offense"])


'CGS  29-35(a)'

In [27]:
df["Statute"] = df["Offense"].apply(statute_code)

In [28]:
statutes = df["Statute"].unique()


In [29]:
def plot_title(ascii_str):
    if pd.isnull(ascii_str): return ascii_str
    return ascii_str[:ascii_str.index("(")].lower().capitalize().strip()
df["Title"] = df["ascii_offense"].apply(plot_title)

In [35]:
def fix(val):
    return str(int(str(val).strip("*").strip().replace(",","")))
def fix_col(col_name):
    df[col_name] = df[col_name].apply(fix).astype(int)
fix_col("Year")
fix_col("Charges")
fix_col("Convictions")

In [36]:
def statute_df(statute):
    return df[df["Statute"] == statute].set_index("Year").sort_index()

In [38]:
df.dtypes

Year              int64
Charges           int64
Convictions       int64
Not Guilty        int64
Nolle            object
Offense          object
Table             int64
ascii_offense    object
Statute          object
Title            object
dtype: object

In [39]:
df["Conviction Rate"] = df["Convictions"] * 100 / df["Charges"]
df.head()

,Year,Charges,Convictions,Not Guilty,Nolle,Offense,Table,ascii_offense,Statute,Title,Conviction Rate
0,2013,4,0,0,4,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,CGS 29-28,Violating pistol permitting requirements,0.000000
1,2014,3,0,0,3,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,CGS 29-28,Violating pistol permitting requirements,0.000000
2,2015,5,2,0,3,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,CGS 29-28,Violating pistol permitting requirements,40.000000
3,2016,7,2,0,5,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,CGS 29-28,Violating pistol permitting requirements,28.571429
4,2017,10,0,0,10,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,1,VIOLATING PISTOL PERMITTING REQUIREMENTS (CGS ...,CGS 29-28,Violating pistol permitting requirements,0.000000


In [40]:
import matplotlib
#matplotlib.use( "agg" )
# matplotlib.use( "macosx" )

%matplotlib inline
# matplotlib.style.use('ggplot')
matplotlib.style.use('mirror00')
import matplotlib.pyplot as plt


Bad key "legend.font.size" on line 37 in
/Users/jakekara/.matplotlib/stylelib/mirror00.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution

Bad key "legend.font.color" on line 38 in
/Users/jakekara/.matplotlib/stylelib/mirror00.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [ ]:
import os

def plot_convictions(tmpdf):
    
    ax = tmpdf["Conviction Rate"].plot.bar()
    
    plt.tight_layout()
    
    try:
        fname = tmpdf.iloc[-1]["Title"]
    except:
        tmpdf.iloc[0]["Title"]
    
    plt.savefig(os.path.join("img", "convictions",  fname + ".png"),
                bbox_inches="tight")
    
    plt.close()

plot_convictions(statute_df(statutes[5]))

In [ ]:
for statute in statutes:
    tmpdf = statute_df(statute)
    
    plot_convictions(tmpdf)
    